 Імпорт і налаштування

In [36]:
import os
import urllib.request
from datetime import datetime
import pandas as pd

BASE_URL = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php"
DATA_DIR = "data"


Функції

In [37]:
def create_data_directory():
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

def download_vhi_data(region_id):
    now = datetime.now().strftime("%d%m%Y%H%M%S")
    filename = f"NOAA_ID{region_id}_{now}.csv"
    filepath = os.path.join(DATA_DIR, filename)

    if not os.path.exists(filepath):
        url = f"{BASE_URL}?country=UKR&provinceID={region_id}&year1=1981&year2=2024&type=Mean"
        try:
            with urllib.request.urlopen(url) as response:
                data = response.read()

            with open(filepath, 'wb') as file:
                file.write(data)

            print(f"[OK] Дані для області {region_id} збережено як {filename}")
        except Exception as e:
            print(f"[ERR] Помилка при завантаженні для області {region_id}: {e}")
    else:
        print(f"[INFO] Файл для області {region_id} вже існує: {filename}. Пропущено завантаження.")

def read_all_data(directory):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_frames = []
    for file in os.listdir(directory):
        if file.startswith("NOAA_ID") and file.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join(directory, file), header=1, names=headers)
                df = df[pd.to_numeric(df['Year'], errors='coerce').notnull()]
                df['Year'] = df['Year'].astype(int)
                df = df[df['VHI'] != -1]
                region_id = int(file.split("ID")[1].split("_")[0])
                df['filename'] = file
                df['area'] = region_id
                all_frames.append(df)
            except Exception as e:
                print(f"[WARN] Пропущено файл {file}: {e}")
    if all_frames:
        return pd.concat(all_frames, ignore_index=True)
    else:
        print("[ERR] Жоден файл не був прочитаний!")
        return pd.DataFrame()

def rename_areas(df):
    index_map = {
        1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
        6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська',
        10: 'Кіровоградська', 11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська',
        14: 'Одеська', 15: 'Полтавська', 16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська',
        19: 'Харківська', 20: 'Херсонська', 21: 'Хмельницька', 22: 'Черкаська',
        23: 'Чернівецька', 24: 'Чернігівська', 25: 'Крим'
    }
    df['region_name'] = df['area'].replace(index_map)
    return df


Статистика та вибірки

In [38]:
def get_vhi_by_year(df, region, year):
    result = df[(df['region_name'] == region) & (df['Year'] == year)][['Week', 'VHI']]
    print(f"\nVHI для області {region} за рік {year}:\n", result)
    return result

def vhi_statistics(df, region, year):
    filtered = df[(df['region_name'] == region) & (df['Year'] == year)]['VHI']
    print(f"\nСтатистика VHI для {region} у {year} році:")
    print(f"  Мінімум: {filtered.min()}")
    print(f"  Максимум: {filtered.max()}")
    print(f"  Середнє: {filtered.mean():.2f}")
    print(f"  Медіана: {filtered.median()}")

def get_vhi_range(df, region, year_start, year_end):
    result = df[(df['region_name'] == region) & (df['Year'].between(year_start, year_end))][['Year', 'Week', 'VHI']]
    print(f"\nVHI для області {region} з {year_start} по {year_end}:\n", result)
    return result

def detect_droughts(df, min_regions=5):
    droughts = df[df['VHI'] < 15]
    drought_years = droughts.groupby('Year')['region_name'].nunique()
    critical_years = drought_years[drought_years >= min_regions]
    print(f"\nРоки з екстремальною посухою (областей >= {min_regions}):")
    for year in critical_years.index:
        regions = droughts[droughts['Year'] == year]['region_name'].unique()
        print(f"  {year}: {len(regions)} областей -> {', '.join(regions)}")

Завантаження та обробка даних

In [39]:
create_data_directory()

existing_files = [f for f in os.listdir(DATA_DIR) if f.startswith("NOAA_ID") and f.endswith('.csv')]
if not existing_files:
    print("Завантаження даних для всіх областей...")
    for i in range(1, 26):
        download_vhi_data(i)
else:
    print("Файли з даними вже існують. Пропущено завантаження.")

print("\nОбробка збережених файлів...")
df = read_all_data(DATA_DIR)
df = rename_areas(df)
df.to_csv("data/final_vhi_data.csv", index=False)


Файли з даними вже існують. Пропущено завантаження.

Обробка збережених файлів...


Виклик функцій 

In [40]:
print("VHI для Київської області за 2023 рік")
get_vhi_by_year(df, 'Київська', 2023)

print("\nСтатистика VHI для Київської області за 2023 рік")
vhi_statistics(df, 'Київська', 2023)

print("\nVHI для Київської області з 2020 по 2023 рік")
get_vhi_range(df, 'Київська', 2020, 2023)

print("\nАналіз років з екстремальною посухою (областей ≥ 5)")
detect_droughts(df)

VHI для Київської області за 2023 рік

VHI для області Київська за рік 2023:
        Week    VHI
54521   1.0  42.94
54522   2.0  41.91
54523   3.0  41.18
54524   4.0  41.64
54525   5.0  41.98
54526   6.0  42.97
54527   7.0  43.24
54528   8.0  43.43
54529   9.0  43.61
54530  10.0  43.30
54531  11.0  43.05
54532  12.0  45.11
54533  13.0  48.32
54534  14.0  51.00
54535  15.0  55.00
54536  16.0  59.67
54537  17.0  64.33
54538  18.0  67.10
54539  19.0  68.99
54540  20.0  67.72
54541  21.0  66.34
54542  22.0  64.63
54543  23.0  61.87
54544  24.0  59.97
54545  25.0  59.93
54546  26.0  58.89
54547  27.0  57.60
54548  28.0  56.35
54549  29.0  55.51
54550  30.0  54.58
54551  31.0  53.19
54552  32.0  50.22
54553  33.0  46.54
54554  34.0  43.08
54555  35.0  39.17
54556  36.0  33.51
54557  37.0  29.27
54558  38.0  27.01
54559  39.0  25.66
54560  40.0  25.65
54561  41.0  25.56
54562  42.0  23.67
54563  43.0  21.96
54564  44.0  20.48
54565  45.0  20.37
54566  46.0  21.21
54567  47.0  23.61
54568  48.